Import Libraries

In [36]:
from pyvis.network import Network
import networkx as nx
import random
from networkx.algorithms.community import greedy_modularity_communities
from networkx.linalg.modularitymatrix import modularity_matrix, directed_modularity_matrix
import numpy as np

Create Helper Functions

In [37]:
# recursive function that will traverse the nodes
def createPaper(network, authors, probStop):
    '''
    Will take network, list of authors, and probStop as input
    '''
    currAuthorID = authors[-1]
    newNeighbors = set(network.neighbors(currAuthorID)).difference(set(authors))

    # base condition: stop at node if probStop hit or there are no new neighbors to traverse
    if random.random() < probStop or len(newNeighbors) == 0:
        return
    
    # create list reprsenting probabilities for the neighboring nodes of the current coauthor
    probs = []
    for neighbor in newNeighbors:
        nData = network.get_edge_data(currAuthorID, neighbor)
        probs.extend([neighbor] * nData["weight"])

    # Select coauthor from neighbors probabilities list
    coauthorID = random.choice(probs)

    # update all edges of coauthors to this new author
    for author in authors:
        # if there is not an edge, create one
        if not network.has_edge(author, coauthorID) and author != coauthorID:
            network.add_edge(author, coauthorID, weight=0, width=1)
        newWeight = network.get_edge_data(author, coauthorID)["weight"] + 1
        #network.update(edges=[ (author, coauthorID, {"weight": newWeight, "width": newWeight//2}) ])
        network.update(edges=[ (author, coauthorID, {"weight": newWeight}) ])

    # call function recursively with coauthor
    authors.append(coauthorID)
    createPaper(network, authors, probStop)


Define initial parameters

In [38]:
# define time steps
timeSteps = 15

# Probabilities
# probability that you generate new author
probNewAuthor = 0.5
# probability that you stop at a given node
probStop = 0.7

# define fields
fieldColors = {"CS": "blue", 
                "Math": "green", 
                "Physics": "red"}
fields = list(fieldColors.keys())

# define initial scholars, will be in form (id, scholarField, color)
scholarField = random.choice(fields)
nodeID = 0


Create Model

In [39]:
network = nx.Graph()
scholarField = random.choice(fields)
network.add_node(nodeID, label=scholarField, color=fieldColors[scholarField])
# go through time steps, add new scholar and paper at each step
for i in range(1, timeSteps):

    # Choose first author, either new scholar or random choice
    currNodes = list(network.nodes())
    authors = [random.choice(currNodes)]

    # with probability, add new author to network set as main author with a coauthor
    if random.random() < probNewAuthor:
        # generate author and field
        scholarField = random.choice(fields)
        nodeID += 1
        author = nodeID
        network.add_node(author, label=scholarField, color=fieldColors[scholarField])

        # generate random coauthor from currNodes, which doesn't have the new node added in
        coauthorID = random.choice(currNodes)
        network.add_edge(author, coauthorID, weight=1, width=1)

        # update authors list
        authors = [author, coauthorID]

    # Add new paper, calling function
    createPaper(network, authors, probStop)

Display Network

In [40]:
nt = Network()
# populates the nodes and edges data structures
nt.from_nx(network)
print(network.edges.data())
nt.show('docs/models/modularity.html')

[(0, 1, {'weight': 4, 'width': 1}), (0, 2, {'weight': 3, 'width': 1}), (0, 6, {'weight': 1, 'width': 1}), (0, 7, {'weight': 1, 'width': 1}), (0, 3, {'weight': 1, 'width': 1}), (0, 8, {'weight': 1, 'width': 1}), (1, 2, {'weight': 2, 'width': 1}), (1, 3, {'weight': 2, 'width': 1}), (1, 5, {'weight': 1, 'width': 1}), (1, 6, {'weight': 1, 'width': 1}), (2, 5, {'weight': 1, 'width': 1}), (2, 7, {'weight': 1, 'width': 1}), (3, 4, {'weight': 1, 'width': 1})]


### Show modularity communities

In [41]:
communities = greedy_modularity_communities(network)
print(communities)
matrix = modularity_matrix(network)
print(matrix)

[frozenset({0, 8, 6, 7}), frozenset({1, 2, 5}), frozenset({3, 4})]
[[-1.38461538 -0.15384615  0.07692308  0.30769231 -0.23076923 -0.46153846
   0.53846154  0.53846154  0.76923077]
 [-0.15384615 -0.96153846  0.23076923  0.42307692 -0.19230769  0.61538462
   0.61538462 -0.38461538 -0.19230769]
 [ 0.07692308  0.23076923 -0.61538462 -0.46153846 -0.15384615  0.69230769
  -0.30769231  0.69230769 -0.15384615]
 [ 0.30769231  0.42307692 -0.46153846 -0.34615385  0.88461538 -0.23076923
  -0.23076923 -0.23076923 -0.11538462]
 [-0.23076923 -0.19230769 -0.15384615  0.88461538 -0.03846154 -0.07692308
  -0.07692308 -0.07692308 -0.03846154]
 [-0.46153846  0.61538462  0.69230769 -0.23076923 -0.07692308 -0.15384615
  -0.15384615 -0.15384615 -0.07692308]
 [ 0.53846154  0.61538462 -0.30769231 -0.23076923 -0.07692308 -0.15384615
  -0.15384615 -0.15384615 -0.07692308]
 [ 0.53846154 -0.38461538  0.69230769 -0.23076923 -0.07692308 -0.15384615
  -0.15384615 -0.15384615 -0.07692308]
 [ 0.76923077 -0.19230769 -0.

In [46]:
# get eigenvalues and vectors of matrix
eVals, eVects = np.linalg.eig(matrix)

# find indices of largest eigenvalue, get corresponding eigenvector
maxEval = max(eVals)
print(maxEval)
index = np.where(eVals == maxEval)
print(index)
maxEvect = eVects[index]
print(maxEvect)

# if no positive eigenvalue, then cannot divide network further because it will not improve modularity

1.1482942981418913
(array([4]),)
[[ 0.20613509 -0.11532401  0.09777809  0.6155276   0.48875356  0.20246698
   0.40372981 -0.27233686  0.27438854]]
